<a href="https://colab.research.google.com/github/amotz1/backet_classification/blob/master/MNISTLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib.request
import gzip 
import shutil

In [ ]:
datapath = '/content/MnistData/'

In [ ]:
if not os.path.exists(datapath):
  os.makedirs(datapath)

In [ ]:
 urls = ['https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz',
         'https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz',
         'https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz',
         'https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz']



In [ ]:
for url in urls:
    filename = url.split('/')[-1]   # GET FILENAME
    
    if os.path.exists(datapath+filename):
        print(filename, ' already exists')  # CHECK IF FILE EXISTS
    else:
        print('Downloading ',filename)
        urllib.request.urlretrieve (url, datapath+filename) # DOWNLOAD FILE
     

print(datapath+filename)

/content/MnistData/t10k-labels-idx1-ubyte.gz


In [ ]:
files = os.listdir(datapath)

In [ ]:
print(files)

['train-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz']


In [ ]:
for file in files:
  if file.endswith('gz'):
    print('extracting file ',file) 
    with gzip.open(datapath+file,'rb') as f_in:
      with open(datapath+file.split('.')[0],'wb') as f_out:
        shutil.copyfileobj(f_in,f_out)

print('extraction completed')
for file in files:
  print('removing archives')
  os.remove(datapath+file)

print('completed to remove archives')



extracting file  train-images-idx3-ubyte.gz
extracting file  t10k-images-idx3-ubyte.gz
extracting file  t10k-labels-idx1-ubyte.gz
extracting file  train-labels-idx1-ubyte.gz
extraction completed
removing archives
removing archives
removing archives
removing archives
completed to remove archives


In [ ]:
import os
import codecs
import numpy as np
import torch

In [ ]:
def get_int(b):
  return int(codecs.encode(b,'hex'), 16)

In [ ]:
mnist_dict = {}

In [ ]:
files = os.listdir(datapath)

In [ ]:
files

['train-labels-idx1-ubyte',
 't10k-labels-idx1-ubyte',
 't10k-images-idx3-ubyte',
 'train-images-idx3-ubyte']

In [ ]:
for file in files:
  if file.endswith('ubyte'):
    print('reading ',file)

    with open(datapath+file, 'rb') as f:
      data = f.read()

      type = get_int(data[:4])
      length = get_int(data[4:8])

      if type == 2051:
        category = 'images'
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])

        parsed = np.frombuffer(data,dtype='uint8',offset = 16)
        parsed = parsed.reshape(length, num_rows, num_cols)

      elif type == 2049:
        category = 'labels'
        parsed = np.frombuffer(data,dtype='uint8',offset = 8)
        parsed = parsed.reshape(length)
      
      else:
        assert False,'unspecified type'
      
      if length == 60000:
        data_set = 'train'

      elif length == 10000:
        data_set = 'test'

      else:
        assert False, 'unspecified length'

      mnist_dict[data_set + '_' + category] = parsed 



      


reading  train-labels-idx1-ubyte
reading  t10k-labels-idx1-ubyte
reading  t10k-images-idx3-ubyte
reading  train-images-idx3-ubyte


In [ ]:
test_images_copy = mnist_dict['test_images'].copy()


In [ ]:
train_images_copy = mnist_dict['train_images'].copy()

In [ ]:
def create_diffuser(M, N, diffuser_mean = 0.5, diffuser_std = 0.5, dim = 1): # Create Diffuser
    diffuser_transmission = np.random.normal(diffuser_mean, diffuser_std, [M, N])
    for i in range(M):
        for j in range(N):
            diffuser_transmission[i, j] = max(diffuser_transmission[i, j], 0)
            diffuser_transmission[i, j] = min(diffuser_transmission[i, j], 1)
    # diffuser_transmission = torch.from_numpy(diffuser_transmission)
    return diffuser_transmission

In [ ]:
diffuser = create_diffuser(28, 28)

In [ ]:
for i in range(len(test_images_copy)):
  test_images_copy[i] = test_images_copy[i]@diffuser 

In [ ]:
for i in range (len(train_images_copy)):
  train_images_copy[i] = train_images_copy[i]@diffuser 

In [ ]:
print((train_images_copy == mnist_dict['train_images']).all())
print((test_images_copy == mnist_dict['test_images']).all())

False
False


In [ ]:
mnist_dict['train_images'] = train_images_copy
mnist_dict['test_images'] = test_images_copy

In [ ]:
mnist_dict['train_labels'][2]

4

In [ ]:
from skimage.io import imsave

In [ ]:
datasets = ['train','test']

In [ ]:
categimport numpy as npkjljl

In [ ]:
for dataset in datasets:   # FOR TRAIN AND TEST SET
    images = mnist_dict[dataset+'_images']   # IMAGES
    labels = mnist_dict[dataset+'_labels']   # LABELS
    no_of_samples = images.shape[0]     # NUBMER OF SAMPLES
    for indx in range (no_of_samples):  # FOR EVERY SAMPLE
        print(dataset, indx)
        image = images[indx]            # GET IMAGE
        label = labels[indx]            # GET LABEL
        if not os.path.exists(datapath+dataset+'/'+str(label)+'/'):    # IF DIRECTORIES DO NOT EXIST THEN 
            os.makedirs (datapath+dataset+'/'+str(label)+'/')       # CREATE TRAIN/TEST DIRECTORY AND CLASS SPECIFIC SUBDIRECTORY
        filenumber = len(os.listdir(datapath+dataset+'/'+str(label)+'/'))  # NUMBER OF FILES IN THE DIRECTORY FOR NAMING THE FILE
        imsave(datapath+dataset+'/'+str(label)+'/%05d.png'%(filenumber),image)  # SAVE T


Streaming output truncated to the last 5000 lines.
test 5000
test 5001
test 5002
test 5003
test 5004
test 5005
test 5006
test 5007
test 5008
test 5009
test 5010
test 5011
test 5012
test 5013
test 5014
test 5015
test 5016
test 5017
test 5018
test 5019
test 5020
test 5021
test 5022
test 5023
test 5024
test 5025
test 5026
test 5027
test 5028
test 5029
test 5030
test 5031
test 5032
test 5033
test 5034
test 5035
test 5036
test 5037
test 5038
test 5039
test 5040
test 5041
test 5042
test 5043
test 5044
test 5045
test 5046
test 5047
test 5048
test 5049
test 5050
test 5051
test 5052
test 5053
test 5054
test 5055
test 5056
test 5057
test 5058
test 5059
test 5060
test 5061
test 5062
test 5063
test 5064
test 5065
test 5066
test 5067
test 5068
test 5069
test 5070
test 5071
test 5072
test 5073
test 5074
test 5075
test 5076
test 5077
test 5078
test 5079
test 5080
test 5081
test 5082
test 5083
test 5084
test 5085
test 5086
test 5087
test 5088
test 5089
test 5090
test 5091
test 5092
test 5093
test 5094

In [ ]:
!git clone https://github.com/amotz1/backet_classification.git

Cloning into 'backet_classification'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 179 (delta 120), reused 114 (delta 55), pack-reused 0
Receiving objects: 100% (179/179), 22.12 KiB | 11.06 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
pwd

'/content'

In [ ]:
cd backet_classification/

/content/backet_classification


In [ ]:
pip install wandb --upgrade

     |████████████████████████████████| 1.8MB 9.4MB/s 
     |████████████████████████████████| 133kB 48.6MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=7008cdc77cbdf8e7a9fa7dab118e98289f3e7bc388ac4faa44fa55aecead64d1
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=9ba7babc0ec9eb09468925330a49aba6b4cdfe4df234daa8f6a0b2cdeec8014b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
!python3 main.py

wandb: Currently logged in as: amotz (use `wandb login --relogin` to force relogin)
2021-07-01 16:52:13.150271: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run dry-night-173
wandb: ⭐️ View project at https://wandb.ai/amotz/backet_classification
wandb: 🚀 View run at https://wandb.ai/amotz/backet_classification/runs/1kaal5ed
wandb: Run data is saved locally in /content/backet_classification/wandb/run-20210701_165211-1kaal5ed
wandb: Run `wandb offline` to turn off syncing.

model.fc.requires_grad  Linear(in_features=512, out_features=10, bias=True)
	 fc.weight
	 fc.bias
CNN(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, paddin